In [65]:
from citadel import Citadel
import json
import pandas as pd
from copy import deepcopy
import pdb

In [54]:
with open('config/citadel_config.json', 'r') as fp:
    citadel_config = json.load(fp)
citadel = Citadel(citadel_config['hostname'], citadel_config['apikey'])

In [10]:
df = pd.read_csv('data/sample_waze.csv')
df.head()

,uuid,waze_timestamp,street,end_node,city,length,delay,speed,level,roadtype,line
0,1833531258,2017-02-03 23:13:00:000,E Palomar St,Santa Andrea St,"Chula Vista, CA",918,194,3.455556,3,2,"LINESTRING (-117.004479 32.621195, -117.003013..."
1,1835749474,2017-02-03 23:13:00:000,Coronado Ave,to I-5 S,"San Diego, CA",408,153,2.088889,4,7,"LINESTRING (-117.088446 32.576476, -117.087761..."
2,1835187546,2017-02-03 23:13:00:000,3rd Ave,L St,"Chula Vista, CA",1006,294,2.686111,3,2,"LINESTRING (-117.068427 32.61198, -117.068756 ..."
3,1800053699,2017-02-03 23:13:00:000,Hoover Ave,W 21st St,"National City, CA",100,-1,0.000000,5,1,"LINESTRING (-117.10495 32.664898, -117.104636 ..."
4,1802056625,2017-02-03 23:13:00:000,Hoover Ave,NaN,"National City, CA",100,-1,0.000000,5,1,"LINESTRING (-117.104636 32.664031, -117.10495 ..."


In [34]:
metadata_columns = ['street', 'city', 'roadtype']
data_columns = ['length', 'delay', 'speed', 'level']

### Column Classification

- Metadata columns: street, city, roadtype
- Datastream columns: length, delay, speed, level (waze_timestamp and line are common across the datastreams)

In [ ]:
unit_map = {
    'traffic_length': 'MI',
    'traffic_delay': 'SEC',
    'traffic_level': 'no_unit',
    'traffic_speed': 'MI-PER-HR',
}
point_type_map = {
    'length': 'traffic_length',
    'delay': 'traffic_delay',
    'level': 'traffic_level',
    'speed': 'traffic_speed'
}
def get_name(srcid, point_type):
    return 'waze_{0}_{1}'.format(srcid, point_type)

In [22]:
srcids = df['uuid'].unique()
srcid = srcids[0]

In [27]:
df.loc[df['uuid']==srcid]

,uuid,waze_timestamp,street,end_node,city,length,delay,speed,level,roadtype,line
0,1833531258,2017-02-03 23:13:00:000,E Palomar St,Santa Andrea St,"Chula Vista, CA",918,194,3.455556,3,2,"LINESTRING (-117.004479 32.621195, -117.003013..."
198,1833531258,2017-02-03 23:18:00:000,E Palomar St,Santa Andrea St,"Chula Vista, CA",721,264,2.255556,3,2,"LINESTRING (-117.002846 32.622301, -117.002729..."
375,1833531258,2017-02-03 23:23:00:000,E Palomar St,Santa Andrea St,"Chula Vista, CA",721,213,2.680556,3,2,"LINESTRING (-117.002846 32.622301, -117.002729..."
571,1833531258,2017-02-03 23:28:00:000,E Palomar St,NaN,"Chula Vista, CA",431,176,2.063889,4,2,"LINESTRING (-117.002846 32.622301, -117.002729..."
739,1833531258,2017-02-03 23:33:00:000,E Palomar St,Santa Andrea St,"Chula Vista, CA",525,157,2.619444,3,2,"LINESTRING (-117.000909 32.622943, -116.998884..."


In [24]:
point = df.loc[df['uuid']==srcid].iloc[0]

In [25]:
point

uuid                                                     1833531258
waze_timestamp                              2017-02-03 23:13:00:000
street                                                 E Palomar St
end_node                                            Santa Andrea St
city                                                Chula Vista, CA
length                                                          918
delay                                                           194
speed                                                       3.45556
level                                                             3
roadtype                                                          2
line              LINESTRING (-117.004479 32.621195, -117.003013...
Name: 0, dtype: object

In [37]:
traffic = df.loc[df['uuid']==srcid].iloc[0]
traffic_metadata = dict((k, traffic[k]) for k in metadata_columns)

In [ ]:
# Structuring Metadata 
metadatas = dict()
for srcid in srcids:
    traffic = df.loc[df['uuid']==srcid].iloc[0]
    traffic = dict((k, traffic[k]) for k in metadata_columns)
    #for srcid in srcids:
    traffic = dict()
    traffic['waze_srcid'] = str(srcid)
    for col in data_columns:
        point_type = 'traffic_' + col
        point = deepcopy(traffic)
        name = get_name(srcid, point_type)
        point['pointType'] = point_type
        point['unit'] = unit_map[point_type]
        point['name'] = name
        metadatas[name] = point

In [74]:
name = 'waze_1830951324_traffic_speed'
point = metadatas[name]

In [ ]:
# Metadata Registration to Citadel
for name, metadata in metadatas.items():
    uuids = citadel.query_points({'name': name})
    if not uuids:
        uuid = citadel.create_point(metadata)
        if not res:
            pdb.set_trace()
    else:
        uuid = uuids[0]
    metadata['uuid'] = uuid

In [ ]:
with open('metadata/waze-metadata', 'w') as fp:
    json.dump(fp, metadatas)

In [ ]:
len(df)

In [ ]:
arrow.get('2017-02-03 23:13:00:000').

In [ ]:
data = []
for _, row in df.iterrows()[0:10]:
    srcid = row['uuid']
    t = arrow.get(row['waze_timestamp']).timestamp * 1000
    line = xxxxxxx
    for data_col in data_columns:
        point_type = point_type_map[data_col]
        name = get_name(srcid, point_type)
        uuid = metadatas[name]['uuid']
        datum = {
            'value': row[data_col],
            'coordinates': line,
            'timestamp': arrow.get(),
            'geometryType': 'line',
            'uuid': uuid,
        }
        data.append(datum)

In [ ]:
res = citadel.post_data(data)